<a href="https://colab.research.google.com/github/ehsan74814/Preprocessing_Data/blob/main/logisticRegression_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings

In [20]:
warnings.filterwarnings('ignore')

In [37]:
df = pd.read_csv("/content/winequalityN.copy.csv")

In [ ]:
df = df.drop(columns='type', axis=1)

In [42]:
correlation_matrix = df.corr()

In [ ]:
mask =np.triu(np.ones_like(correlation_matrix, dtype=bool))
plt.figure(figsize=(15,8))
sns.heatmap(correlation_matrix,mask= mask, annot=True, cmap="coolwarm", fmt=".2f", cbar=True )
plt.show()